## Lineární a polynomiální regrese pro stanovení cen dýní - Lekce 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Úvod

Doposud jste prozkoumali, co je regrese, pomocí vzorových dat získaných z datasetu cen dýní, který budeme používat v průběhu této lekce. Také jste ji vizualizovali pomocí `ggplot2`.💪

Nyní jste připraveni ponořit se hlouběji do regrese pro strojové učení. V této lekci se dozvíte více o dvou typech regrese: *základní lineární regrese* a *polynomiální regrese*, spolu s některými matematickými základy těchto technik.

> V celém tomto kurzu předpokládáme minimální znalosti matematiky a snažíme se ji zpřístupnit studentům z jiných oborů, takže sledujte poznámky, 🧮 upozornění, diagramy a další učební nástroje, které vám pomohou s pochopením.

#### Příprava

Připomeňme si, že tato data načítáte, abyste si mohli klást otázky.

-   Kdy je nejlepší čas na nákup dýní?

-   Jakou cenu mohu očekávat za bednu mini dýní?

-   Mám je koupit v půlkoších nebo v krabici o velikosti 1 1/9 bušlu? Pojďme se do těchto dat ponořit hlouběji.

V předchozí lekci jste vytvořili `tibble` (moderní přepracování datového rámce) a naplnili jej částí původního datasetu, standardizovali ceny podle bušlu. Tímto způsobem jste však byli schopni shromáždit pouze asi 400 datových bodů a pouze pro podzimní měsíce. Možná můžeme získat trochu více detailů o povaze dat jejich důkladnějším vyčištěním? Uvidíme... 🕵️‍♀️

Pro tento úkol budeme potřebovat následující balíčky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekce balíčků pro R](https://www.tidyverse.org/packages), která je navržena tak, aby byla datová věda rychlejší, jednodušší a zábavnější!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [rámec balíčků](https://www.tidymodels.org/packages/) pro modelování a strojové učení.

-   `janitor`: [balíček janitor](https://github.com/sfirke/janitor) poskytuje jednoduché nástroje pro zkoumání a čištění špinavých dat.

-   `corrplot`: [balíček corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) poskytuje vizuální nástroj pro zkoumání korelační matice, který podporuje automatické přeskupování proměnných, aby pomohl odhalit skryté vzory mezi proměnnými.

Můžete je nainstalovat pomocí:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Níže uvedený skript zkontroluje, zda máte balíčky potřebné k dokončení tohoto modulu, a v případě jejich absence je pro vás nainstaluje.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Později načteme tyto skvělé balíčky a zpřístupníme je v naší aktuální R relaci. (Toto je pouze pro ilustraci, `pacman::p_load()` to už za vás udělal)

## 1. Přímka lineární regrese

Jak jste se naučili v Lekci 1, cílem cvičení lineární regrese je být schopen vykreslit *přímku* *nejlepšího* *přizpůsobení*, která:

-   **Ukazuje vztahy mezi proměnnými**. Zobrazuje vztah mezi proměnnými.

-   **Umožňuje predikce**. Poskytuje přesné předpovědi, kam by nový datový bod spadal ve vztahu k této přímce.

K vykreslení tohoto typu přímky používáme statistickou techniku nazvanou **Regrese nejmenších čtverců**. Termín `nejmenší čtverce` znamená, že všechny datové body obklopující regresní přímku jsou umocněny na druhou a poté sečteny. Ideálně je tento konečný součet co nejmenší, protože chceme mít nízký počet chyb, tedy `nejmenší čtverce`. Přímka nejlepšího přizpůsobení je tedy ta, která nám dává nejnižší hodnotu součtu čtverců chyb – odtud název *regrese nejmenších čtverců*.

Děláme to proto, že chceme modelovat přímku, která má nejmenší kumulativní vzdálenost od všech našich datových bodů. Termíny také umocňujeme na druhou před jejich sečtením, protože nás zajímá jejich velikost, nikoli směr.

> **🧮 Ukažte mi matematiku**
>
> Tato přímka, nazývaná *přímka nejlepšího přizpůsobení*, může být vyjádřena [rovnicí](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` je '`vysvětlující proměnná` nebo `prediktor`'. `Y` je '`závislá proměnná` nebo `výsledek`'. Sklon přímky je `b` a `a` je průsečík s osou y, což označuje hodnotu `Y`, když `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "sklon = $y/x$")
    Infografika od Jen Looper
>
> Nejprve vypočítejte sklon `b`.
>
> Jinými slovy, a s odkazem na původní otázku o datech dýní: "předpovězte cenu dýně za bušl podle měsíce", `X` by označovalo cenu a `Y` by označovalo měsíc prodeje.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografika od Jen Looper
> 
> Vypočítejte hodnotu Y. Pokud platíte kolem 4 $, musí být duben!
>
> Matematika, která vypočítává přímku, musí ukázat sklon přímky, který také závisí na průsečíku, tedy na tom, kde se `Y` nachází, když `X = 0`.
>
> Metodu výpočtu těchto hodnot si můžete prohlédnout na webu [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Navštivte také [tento kalkulátor nejmenších čtverců](https://www.mathsisfun.com/data/least-squares-calculator.html), abyste viděli, jak hodnoty čísel ovlivňují přímku.

Není to tak děsivé, že? 🤓

#### Korelace

Ještě jeden termín, který je třeba pochopit, je **koeficient korelace** mezi danými proměnnými X a Y. Pomocí bodového grafu můžete tento koeficient rychle vizualizovat. Graf s body uspořádanými do úhledné přímky má vysokou korelaci, ale graf s body rozptýlenými všude mezi X a Y má nízkou korelaci.

Dobrý model lineární regrese bude takový, který má vysoký (blíže k 1 než k 0) koeficient korelace při použití metody regrese nejmenších čtverců s regresní přímkou.


## **2. Tanec s daty: vytvoření datového rámce pro modelování**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>


<!--![Ilustrace od \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Načtěte potřebné knihovny a dataset. Převést data na datový rámec obsahující podmnožinu dat:

-   Získat pouze dýně oceněné podle bušlu

-   Převést datum na měsíc

-   Vypočítat cenu jako průměr vysokých a nízkých cen

-   Převést cenu tak, aby odrážela ocenění podle množství v bušlech

> Tyto kroky jsme pokryli v [předchozí lekci](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Ve duchu čistého dobrodružství pojďme prozkoumat [`balíček janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), který poskytuje jednoduché funkce pro zkoumání a čištění nepořádných dat. Například se podívejme na názvy sloupců našich dat:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Můžeme to udělat lépe. Pojďme tyto názvy sloupců `friendR` převést na konvenci [snake_case](https://en.wikipedia.org/wiki/Snake_case) pomocí `janitor::clean_names`. Chcete-li se o této funkci dozvědět více: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Mnoho tidyR 🧹! Teď tanec s daty pomocí `dplyr`, stejně jako v předchozí lekci! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Dobrá práce!👌 Nyní máte čistou a upravenou datovou sadu, na které můžete postavit svůj nový regresní model!

Co takhle bodový graf?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Bodový graf nám připomíná, že máme data pouze za měsíce od srpna do prosince. Pravděpodobně budeme potřebovat více dat, abychom mohli vyvozovat závěry lineárním způsobem.

Podívejme se znovu na naše modelovací data:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Co kdybychom chtěli předpovědět `cenu` dýně na základě sloupců `město` nebo `balení`, které jsou typu znak? Nebo ještě jednodušeji, jak bychom mohli najít korelaci (která vyžaduje, aby oba její vstupy byly číselné) například mezi `balení` a `cenou`? 🤷🤷

Modely strojového učení fungují nejlépe s číselnými vlastnostmi než s textovými hodnotami, takže je obvykle potřeba převést kategorické vlastnosti na číselné reprezentace.

To znamená, že musíme najít způsob, jak přeformátovat naše prediktory, aby je model mohl efektivně využít, což je proces známý jako `feature engineering`.


## 3. Předzpracování dat pro modelování s balíčkem recipes 👩‍🍳👨‍🍳

Aktivity, které upravují hodnoty prediktorů tak, aby je model mohl efektivněji využít, se označují jako `feature engineering`.

Různé modely mají různé požadavky na předzpracování. Například metoda nejmenších čtverců vyžaduje `kódování kategoriálních proměnných`, jako jsou měsíc, odrůda a city_name. To jednoduše zahrnuje `převod` sloupce s `kategoriálními hodnotami` na jeden nebo více `numerických sloupců`, které nahradí původní.

Například, pokud vaše data obsahují následující kategoriální proměnnou:

|  město   |
|:--------:|
| Denver   |
| Nairobi  |
|  Tokio   |

Můžete použít *ordinální kódování*, které nahradí každou kategorii unikátní celočíselnou hodnotou, například takto:

| město |
|:-----:|
|   0   |
|   1   |
|   2   |

A přesně to uděláme s našimi daty!

V této části prozkoumáme další úžasný balíček z Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - který je navržen tak, aby vám pomohl předzpracovat data **před** trénováním modelu. Základem balíčku je objekt zvaný recept, který definuje, jaké kroky by měly být aplikovány na datovou sadu, aby byla připravena pro modelování.

Teď si vytvoříme recept, který připraví naše data pro modelování tím, že nahradí unikátní celočíselnou hodnotu za všechny pozorování ve sloupcích prediktorů:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Skvělé! 👏 Právě jsme vytvořili náš první recept, který určuje výsledek (cenu) a jeho odpovídající prediktory, přičemž všechny sloupce prediktorů by měly být zakódovány jako sada celých čísel 🙌! Pojďme si to rychle rozebrat:

-   Volání `recipe()` s formulí říká receptu *role* proměnných, přičemž jako referenční data používá `new_pumpkins`. Například sloupec `price` byl přiřazen roli `outcome`, zatímco ostatní sloupce byly přiřazeny roli `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` určuje, že všechny prediktory by měly být převedeny na sadu celých čísel, přičemž číslování začíná od 0.

Jsme si jistí, že vás možná napadá něco jako: "To je tak skvělé!! Ale co když budu potřebovat potvrdit, že recepty dělají přesně to, co od nich očekávám? 🤔"

To je skvělá myšlenka! Vidíte, jakmile je váš recept definován, můžete odhadnout parametry potřebné k předzpracování dat a poté extrahovat zpracovaná data. Obvykle to nemusíte dělat, když používáte Tidymodels (za chvíli uvidíme běžnou konvenci -\> `workflows`), ale může se to hodit, když chcete provést nějakou kontrolu správnosti a potvrdit, že recepty dělají to, co očekáváte.

K tomu budete potřebovat dvě další funkce: `prep()` a `bake()`, a jako vždy vám naši malí R kamarádi od [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) pomohou lépe pochopit tento proces!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): odhaduje potřebné parametry z trénovací sady, které mohou být později aplikovány na jiné datové sady. Například, pro daný sloupec prediktoru, která pozorování budou přiřazena jako celé číslo 0, 1, 2 atd.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): vezme připravený recept a aplikuje operace na jakoukoli datovou sadu.

To znamená, pojďme připravit a upéct naše recepty, abychom si skutečně ověřili, že v zákulisí budou sloupce prediktorů nejprve zakódovány, než bude model natrénován.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Hurá! 🥳 Zpracovaná data `baked_pumpkins` mají všechny své prediktory zakódované, což potvrzuje, že kroky předzpracování definované jako náš recept budou fungovat podle očekávání. To sice ztěžuje čtení, ale pro Tidymodels je to mnohem srozumitelnější! Udělejte si chvíli na to, abyste zjistili, která pozorování byla namapována na odpovídající celé číslo.

Za zmínku také stojí, že `baked_pumpkins` je datový rámec, na kterém můžeme provádět výpočty.

Například se pokusme najít dobrou korelaci mezi dvěma body vašich dat, abychom potenciálně vytvořili dobrý prediktivní model. K tomu použijeme funkci `cor()`. Zadejte `?cor()`, abyste se dozvěděli více o této funkci.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Jak se ukazuje, mezi městem a cenou existuje pouze slabá korelace. Nicméně mezi balením a jeho cenou je o něco lepší korelace. To dává smysl, že? Obvykle platí, že čím větší je krabice s produkty, tím vyšší je cena.

Když už jsme u toho, pojďme také zkusit vizualizovat korelační matici všech sloupců pomocí balíčku `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Mnohem lepší.

Dobrá otázka, kterou si teď můžeme položit, je: '`Jakou cenu mohu očekávat za daný balíček dýní?`' Pojďme se na to podívat!

> Poznámka: Když **`bake()`** připravený recept **`pumpkins_prep`** s **`new_data = NULL`**, získáte zpracovaná (tj. zakódovaná) tréninková data. Pokud byste měli jinou datovou sadu, například testovací sadu, a chtěli byste vidět, jak by ji recept předzpracoval, jednoduše byste použili **`bake`** na **`pumpkins_prep`** s **`new_data = test_set`**.

## 4. Vytvořte lineární regresní model

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Nyní, když jsme vytvořili recept a skutečně potvrdili, že data budou správně předzpracována, pojďme vytvořit regresní model, který odpoví na otázku: `Jakou cenu mohu očekávat u daného balení dýní?`

#### Natrénujte lineární regresní model pomocí trénovací sady

Jak jste si už možná všimli, sloupec *price* je proměnná `výsledek`, zatímco sloupec *package* je proměnná `prediktor`.

K tomu nejprve rozdělíme data tak, že 80 % půjde do trénovací sady a 20 % do testovací sady, poté definujeme recept, který zakóduje sloupec prediktoru do sady celých čísel, a následně vytvoříme specifikaci modelu. Recept nebudeme připravovat ani péct, protože už víme, že data předzpracuje podle očekávání.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Skvělá práce! Teď, když máme recept a specifikaci modelu, musíme najít způsob, jak je spojit do jednoho objektu, který nejprve předzpracuje data (prep+bake na pozadí), přizpůsobí model na předzpracovaná data a také umožní případné aktivity po zpracování. Jak se vám to líbí pro klid duše!🤩

V Tidymodels se tento praktický objekt nazývá [`workflow`](https://workflows.tidymodels.org/) a pohodlně uchovává vaše modelovací komponenty! To je to, co bychom v *Pythonu* nazvali *pipelines*.

Tak pojďme všechno spojit do workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Navíc lze pracovní postup přizpůsobit/vytrénovat podobně jako model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Z výstupu modelu můžeme vidět koeficienty naučené během trénování. Tyto koeficienty představují parametry přímky nejlepšího přizpůsobení, která nám poskytuje nejnižší celkovou chybu mezi skutečnou a předpovězenou proměnnou.

#### Vyhodnocení výkonu modelu pomocí testovací sady

Je čas zjistit, jak si model vedl 📏! Jak na to?

Teď, když jsme model natrénovali, můžeme ho použít k předpovědím pro testovací_sadu pomocí `parsnip::predict()`. Poté můžeme tyto předpovědi porovnat se skutečnými hodnotami štítků, abychom vyhodnotili, jak dobře (nebo ne!) model funguje.

Začněme tím, že vytvoříme předpovědi pro testovací sadu a poté připojíme sloupce k testovací sadě.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Ano, právě jste natrénovali model a použili ho k provádění predikcí! 🔮 Je to dobré? Pojďme zhodnotit výkon modelu!

V Tidymodels to děláme pomocí `yardstick::metrics()`! Pro lineární regresi se zaměříme na následující metriky:

-   `Root Mean Square Error (RMSE)`: Druhá odmocnina [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Poskytuje absolutní metriku ve stejné jednotce jako cílová proměnná (v tomto případě cena dýně). Čím menší hodnota, tím lepší model (zjednodušeně řečeno, představuje průměrnou cenu, o kterou se predikce mýlí!).

-   `Coefficient of Determination (obvykle známý jako R-squared nebo R2)`: Relativní metrika, u které platí, že čím vyšší hodnota, tím lepší přizpůsobení modelu. V podstatě tato metrika ukazuje, kolik rozptylu mezi predikovanými a skutečnými hodnotami cílové proměnné je model schopen vysvětlit.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Tam jde výkon modelu. Podívejme se, zda můžeme získat lepší představu vizualizací bodového grafu balíčku a ceny, a poté použijeme předpovědi k překrytí čáry nejlepšího přizpůsobení.

To znamená, že budeme muset připravit a zpracovat testovací sadu, abychom zakódovali sloupec balíčku a následně jej spojili s předpověďmi vytvořenými naším modelem.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Skvělé! Jak můžete vidět, lineární regresní model příliš dobře negeneralizuje vztah mezi balíčkem a jeho odpovídající cenou.

🎃 Gratulace, právě jste vytvořili model, který může pomoci předpovědět cenu několika druhů dýní. Vaše sváteční dýňové pole bude nádherné. Ale pravděpodobně můžete vytvořit ještě lepší model!

## 5. Vytvořte model polynomiální regrese

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Někdy naše data nemusí mít lineární vztah, ale přesto chceme předpovědět výsledek. Polynomická regrese nám může pomoci provádět předpovědi pro složitější nelineární vztahy.

Vezměme si například vztah mezi balením a cenou v našem datovém souboru s dýněmi. Zatímco někdy existuje lineární vztah mezi proměnnými - čím větší je dýně podle objemu, tím vyšší je cena - někdy tyto vztahy nelze vykreslit jako rovinu nebo přímku.

> ✅ Zde jsou [další příklady](https://online.stat.psu.edu/stat501/lesson/9/9.8) dat, která by mohla využít polynomickou regresi
>
> Podívejte se znovu na vztah mezi odrůdou a cenou v předchozím grafu. Zdá se, že tento bodový graf by měl být nutně analyzován pomocí přímky? Možná ne. V tomto případě můžete zkusit polynomickou regresi.
>
> ✅ Polynomy jsou matematické výrazy, které mohou obsahovat jednu nebo více proměnných a koeficientů.

#### Natrénujte model polynomické regrese pomocí trénovací sady

Polynomická regrese vytváří *zakřivenou čáru*, která lépe odpovídá nelineárním datům.

Podívejme se, zda polynomický model bude lépe fungovat při předpovědích. Budeme postupovat podobným způsobem jako dříve:

-   Vytvořte recept, který určuje kroky předzpracování, jež by měly být provedeny na našich datech, aby byla připravena pro modelování, tj. kódování prediktorů a výpočet polynomů stupně *n*

-   Vytvořte specifikaci modelu

-   Spojte recept a specifikaci modelu do pracovního postupu

-   Vytvořte model přizpůsobením pracovního postupu

-   Vyhodnoťte, jak dobře model funguje na testovacích datech

Pojďme se do toho pustit!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Vyhodnocení výkonu modelu

👏👏 Vytvořili jste polynomiální model, pojďme udělat předpovědi na testovací sadě!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Hurá, pojďme vyhodnotit, jak si model vedl na testovací sadě pomocí `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Mnohem lepší výkon.

`rmse` kleslo z přibližně 7 na přibližně 3, což naznačuje snížení chyby mezi skutečnou cenou a předpovězenou cenou. Můžete to *volně* interpretovat tak, že průměrně jsou nesprávné předpovědi chybné o zhruba 3 $. `rsq` vzrostlo z přibližně 0,4 na 0,8.

Všechny tyto metriky ukazují, že polynomiální model funguje mnohem lépe než lineární model. Skvělá práce!

Podívejme se, jestli to můžeme vizualizovat!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Můžete vidět zakřivenou čáru, která lépe odpovídá vašim datům! 🤩

Můžete ji udělat ještě hladší tím, že předáte polynomiální formuli funkci `geom_smooth` takto:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Podobně jako hladká křivka!🤩

Takto byste mohli vytvořit novou předpověď:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Predpověď pomocí `polynomial model` dává smysl, vzhledem k bodovým grafům `price` a `package`! A pokud je tento model lepší než ten předchozí, při pohledu na stejná data budete muset počítat s rozpočtem na tyto dražší dýně!

🏆 Skvělá práce! Vytvořili jste dva regresní modely během jedné lekce. V závěrečné části o regresi se naučíte o logistické regresi pro určení kategorií.

## **🚀Výzva**

Vyzkoušejte několik různých proměnných v tomto notebooku a zjistěte, jak korelace odpovídá přesnosti modelu.

## [**Kvíz po přednášce**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Přehled & Samostudium**

V této lekci jsme se naučili o lineární regresi. Existují i další důležité typy regrese. Přečtěte si o technikách Stepwise, Ridge, Lasso a Elasticnet. Skvělý kurz pro další studium je [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Pokud se chcete dozvědět více o používání úžasného frameworku Tidymodels, podívejte se na následující zdroje:

-   Web Tidymodels: [Začněte s Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn a Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **DĚKUJEME:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) za vytvoření úžasných ilustrací, které činí R přívětivějším a poutavějším. Další ilustrace najdete v její [galerii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro kritické informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
